<a href="https://colab.research.google.com/github/NewCodeLearner/DeepLearning_PyTorch/blob/main/SimpleNeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Creating simple neural network

In [2]:
import torch
import torch.nn as nn
import torch.functional as F

In [14]:
#create a model class that inherits the nn.module class

class Model (nn.Module):
    #input layer ( 4 features of the flower)
    #hidden layer 1 (8 neurons)
    #hidden layer 2 (8 neurons)
    #output ( 3 classes of iris flower)

    # we define the neural network inside the __init__ function
    # and then we define forward method to link the layers output

    def __init__(self,input_features=4,hidden_layer1=8,
                 hidden_layer2=8,out_features=3):
        super().__init__() #instantiate nn.module
        self.fc1 = nn.Linear(input_features,hidden_layer1)
        self.fc2 = nn.Linear(hidden_layer1,hidden_layer2)
        self.out = nn.Linear(hidden_layer2,out_features)

    #feed forward
    def forward(self,x):
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.out(x)
        return x



In [15]:
#pick a manual seed for randomization
torch.manual_seed(42)
#create  instance of model

model = Model()